In [1]:
import arcpy 
import os
from arcpy import env
from arcpy.sa import *

In [2]:
# paths to grids 
cesm2_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\cesm2_base_grid"
era5_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\era5_base_grid"
solar_area = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\resourceAssessment.gdb\\InclusionArea_Solar_SL1_ZRM_filters_dissolved"
wind_area = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\data\\resourceAssessment.gdb\\InclusionArea_Wind_SL1_ZRM_filters_dissolved"

# set output workspace and environment
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data"
centroid_workspace = os.path.join(mainOutputFolder, "scratch.gdb")
arcpy.env.workspace = centroid_workspace
arcpy.env.overwriteOutput = True

In [3]:
def getCentroid(grid_fc, cpa_fc):

    # Select all grid cells that overlap with our CPAs
    grid_cells = arcpy.management.SelectLayerByLocation(grid_fc, 'INTERSECT', 
                                                          cpa_fc, 0, 
                                                          'NEW_SELECTION') # we will only use wind cpa because that excludes less land

    # get name of input grid
    grid_name = os.path.splitext(os.path.basename(grid_fc))[0]

    # get name of area
    area_name = os.path.splitext(os.path.basename(cpa_fc))[0]

    # Set ouput feature class name
    output_name = "{}_centroid".format(grid_name) 
    output_path = os.path.join(centroid_workspace, output_name)

    # Convert polygon feature class to point feature class, with the point representing the CENTROID of the feature
    centroid_fc = arcpy.management.FeatureToPoint(grid_cells, output_path, "CENTROID")

    print(f'{output_path}')
    print("centroid saved to {}".format(output_path))

In [4]:
# get list for grids and areas
grid_list = [cesm2_grid]
area_list = [wind_area]

In [5]:
for grid in grid_list:
    for area in area_list:
        getCentroid(grid, area)

C:\Users\Zachary\ASSET\Transmission\analysis\data\scratch.gdb\cesm2_base_grid_centroid
centroid saved to C:\Users\Zachary\ASSET\Transmission\analysis\data\scratch.gdb\cesm2_base_grid_centroid


In [6]:
# Look at the fields of the created feature class
sample_fc = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\scratch.gdb\\cesm2_base_grid_centroid"
fields = [field.name for field in arcpy.ListFields(sample_fc)]
with arcpy.da.SearchCursor(sample_fc, fields) as cursor:
    for row in cursor:
        print(row)

(1, (-2130726.966, 3164618.7874999996), 33.0, 34)
(2, (-2161939.498400001, 3065529.8954000007), 34.0, 35)
(3, (-2319312.3902000003, 2565925.8806), 39.0, 40)
(4, (-2350966.1114000008, 2465436.3556999993), 40.0, 41)
(5, (-2414366.9191999994, 2264160.8976000007), 42.0, 43)
(6, (-2041572.1412000004, 3137178.6341999993), 65.0, 66)
(7, (-2071478.6655000001, 3037687.7782000005), 66.0, 67)
(8, (-2101484.8751, 2937865.2936000004), 67.0, 68)
(9, (-2131578.3709999993, 2837752.429199999), 68.0, 69)
(10, (-2161747.3067000005, 2737388.5950000007), 69.0, 70)
(11, (-2191980.3577999994, 2636811.4639999997), 70.0, 71)
(12, (-2222266.6927000005, 2536057.0702), 71.0, 72)
(13, (-2252595.9452, 2435159.899599999), 72.0, 73)
(14, (-2282958.187999999, 2334152.9771), 73.0, 74)
(15, (-2313343.9088000003, 2233067.947899999), 74.0, 75)
(16, (-2343743.9866000004, 2131935.1548999995), 75.0, 76)
(17, (-2374149.6702999994, 2030783.7106999997), 76.0, 77)
(18, (-1952064.1108999997, 3110913.5009000003), 97.0, 98)
(19, (-

In [7]:
def getFeatureClasses(input_workspace):
    previous_workspace = arcpy.env.workspace
    try:
        arcpy.env.workspace = input_workspace
        feature_class_list = arcpy.ListFeatureClasses()
        return feature_class_list
    finally:
        arcpy.env.workspace = previous_workspace

In [8]:
# set path to input workspace for transmission lines
transmission_lines = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\TransmissionLines.gdb"


In [9]:
voltage_class_list = getFeatureClasses(transmission_lines)
voltage_class_list

['voltage_100_161',
 'voltage_dc',
 'voltage_345',
 'voltage_500',
 'voltage_under_100',
 'voltage_735_and_above',
 'voltage_220_287',
 'voltage_not_available',
 'Electric_Power_Transmission_Lines_project']

In [10]:
# get centroid points 
input_grid_list = arcpy.ListFeatureClasses('*cesm2_base_grid_centroid')
input_grid_list

['cesm2_base_grid_centroid']

In [11]:
for input_grid in input_grid_list:
    for voltage_class in voltage_class_list:

        # Get the full path for the voltage path
        voltage_class_path = os.path.join(transmission_lines, voltage_class)

        # Set output feature class name
        near_name = "{}_{}_near_analysis_twogeos".format(input_grid, voltage_class)
        near_path = os.path.join(centroid_workspace, near_name)

        # Copy input grid
        arcpy.CopyFeatures_management(input_grid, near_path)

        try:
            # Perform near analysis on copied input grid to find the two nearest geometries
            near_table = arcpy.GenerateNearTable_analysis(near_path, voltage_class_path, 'in_memory\\near_table', search_radius = "", location = "LOCATION", angle = "NO_ANGLE", closest = "ALL", closest_count = 2)

            # Extract NEAR_X, NEAR_Y, and NEAR_FID from the near table
            with arcpy.da.SearchCursor(near_table, ['IN_FID', 'NEAR_X', 'NEAR_Y', 'NEAR_FID', 'NEAR_DIST', 'NEAR_RANK']) as cursor:
                for row in cursor:
                    in_fid = row[0]
                    near_x = row[1]
                    near_y = row[2]
                    near_fid_value = row[3]
                    near_dist = row[4]
                    near_rank = row[5]

                    # Update the point feature class with NEAR_X, NEAR_Y, and NEAR_FID for the current NEAR_RANK
                    update_fields = [f'NEAR_X_{near_rank}', f'NEAR_Y_{near_rank}', f'NEAR_FID_{near_rank}', f'NEAR_DIST_{near_rank}']

                    # Check if the fields exist, and if not, create them
                    existing_fields = [field.name for field in arcpy.ListFields(near_path)]
                    for field_name in update_fields:
                        if field_name not in existing_fields:
                            arcpy.AddField_management(near_path, field_name, "DOUBLE")
                
                    with arcpy.da.UpdateCursor(near_path, update_fields, f'OBJECTID = {in_fid}') as update_cursor:
                        for update_row in update_cursor:
                            update_row[0] = near_x
                            update_row[1] = near_y
                            update_row[2] = near_fid_value
                            update_row[3] = near_dist
                            update_cursor.updateRow(update_row)
            
            # Clean up the near table and line layer
            arcpy.Delete_management(near_table)

            # get geoprocessing messages
            print(arcpy.GetMessages())

        except arcpy.ExecuteError:
            print(arcpy.GetMessages(2))
        except Exception as err:
            print(err.args[0])

Start Time: Tuesday, September 26, 2023 8:09:18 AM
Succeeded at Tuesday, September 26, 2023 8:09:18 AM (Elapsed Time: 0.00 seconds)
Start Time: Tuesday, September 26, 2023 8:10:12 AM
Succeeded at Tuesday, September 26, 2023 8:10:12 AM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, September 26, 2023 8:11:09 AM
Succeeded at Tuesday, September 26, 2023 8:11:09 AM (Elapsed Time: 0.00 seconds)
Start Time: Tuesday, September 26, 2023 8:12:05 AM
Succeeded at Tuesday, September 26, 2023 8:12:05 AM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, September 26, 2023 8:13:01 AM
Succeeded at Tuesday, September 26, 2023 8:13:01 AM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, September 26, 2023 8:13:57 AM
Succeeded at Tuesday, September 26, 2023 8:13:57 AM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, September 26, 2023 8:14:54 AM
Succeeded at Tuesday, September 26, 2023 8:14:54 AM (Elapsed Time: 0.01 seconds)
Start Time: Tuesday, September 26, 2023 8:15:50 AM
Succeeded at Tuesday, Sep